In [4]:
import pandas as pd


df = pd.read_csv(r'../joined_dataset.csv')

#PREVENT DUPLICATE CSVs
# #remove duplicates by title and ISBNs
# def remove_duplicate_books(top_books):
#     duplicate_isbns = set()
#     duplicate_titles = set()
#     unique_books = []
    
#     for book in top_books:
#         isbn, title = book[0], book[1]
        
#         if isbn not in duplicate_isbns and title not in duplicate_titles:
#             duplicate_isbns.add(isbn)
#             duplicate_titles.add(title)
#             unique_books.append(book)
    
#     return unique_books

# def get_similar_users_books(book_isbn, df):
#     #Get users who rated the selected book highly
#     similar_users = df[(df['ISBN'] == book_isbn) & (df['Book-Rating'] >= 8)]['User-ID'].unique()

#     #Get 20 books rated by similar users
#     top_books = []
#     for user in similar_users:
#         user_books = df[df['User-ID'] == user].sort_values(by='Book-Rating', ascending=False).head(20)
#         top_books.extend(user_books[['ISBN', 'Book-Title', 'Book-Author']].values.tolist())

#     #remove duplicates by title and ISBNs
#     unique_books = remove_duplicate_books(top_books)

#     return unique_books 


In [18]:
# #made csv for hyperion, death on the nile, and romeo and juliet
# book_title = 'Romeo and Juliet'


# book_data = df[df['Book-Title'].str.contains(book_title, case=False, na=False)].head(1)

# if book_data.empty:
#     print(f"Book '{book_title}' not found!")
# else:  

#     book_isbn = book_data['ISBN'].values[0]

#     #get unique books that are rated by similar users
#     unique_books = get_similar_users_books(book_isbn, df)

#     books_to_recommend_me = pd.DataFrame(unique_books, columns=['ISBN', 'Book-Title', 'Book-Author'])

#     #add empty column to manually input book rating
#     books_to_recommend_me['Book-Rating'] = ''

#     print(len(unique_books))

#     for book in unique_books:
#         print(f"Book: {book[1]}, Author: {book[2]}")
        
#     books_to_recommend_me.to_csv(r'./accuracy_check/romeo_and_juliet_books.csv', index=False)



26
Book: HAMLET, Author: WILLIAM SHAKESPEARE
Book: ISLAND OF THE SEQUINED LOVE NUN, Author: CHRISTOPHER MOORE
Book: SATI, Author: CHRISTOPHER PIKE
Book: THE COLOR PURPLE, Author: ALICE WALKER
Book: STORYVILLE, Author: LOIS BATTLE
Book: HAUNTED HEARTLAND, Author: BETH SCOTT
Book: COMPANIONS OF THE NIGHT (LAUREL-LEAF BOOKS), Author: VIVIAN VANDE VELDE
Book: ORANGES ARE NOT THE ONLY FRUIT, Author: JEANETTE WINTERSON
Book: ROMEO AND JULIET (THE PELICAN SHAKESPEARE), Author: WILLIAM SHAKESPEARE
Book: THE ISLAND OF LOST MAPS : A TRUE STORY OF CARTOGRAPHIC CRIME, Author: MILES HARVEY
Book: LOSING JESSICA, Author: ROBBY DEBOER
Book: LILITH (HEELEY, D. A. DARKNESS AND LIGHT, BK. 1.), Author: D. A. HEELEY
Book: DREAMS IN THE GOLDEN COUNTRY: THE DIARY OF ZIPPORAH FELDMAN, A JEWISH IMMIGRANT GIRL, NEW YORK CITY, 1903 (DEAR AMERICA), Author: KATHRYN LASKY
Book: THE SECOND BEND IN THE RIVER (POINT), Author: ANN RINALDI
Book: RENASCENCE, AND OTHER POEMS (DOVER THRIFT EDITIONS), Author: EDNA ST. VINCE

In [5]:
my_hyperion_ratings = pd.read_csv('hyperion_books.csv')
my_death_on_the_nile_ratings = pd.read_csv('death_on_the_nile_books.csv')
my_romeo_and_juliet_ratings = pd.read_csv('romeo_and_juliet_books.csv')

#combine csv files into one
my_ratings = pd.concat([my_hyperion_ratings, my_death_on_the_nile_ratings, my_romeo_and_juliet_ratings])

#add user-id
if 'User-ID' not in my_ratings.columns:
    my_ratings['User-ID'] = 999999


#add my ratings to joined_dataset
df2 = pd.concat([df, my_ratings])
df2.to_csv('joined_dataset_with_me.csv', index=False)


In [6]:
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split, cross_validate

df2 = pd.read_csv('joined_dataset_with_me.csv')
reader = Reader(rating_scale=(1, 10))

#load updated dataframe into surprise dataset object
data = Dataset.load_from_df(df2[['User-ID', 'ISBN', 'Book-Rating']], reader)

trainset, testset = train_test_split(data, test_size=0.25)

model2 = SVD()
model2.fit(trainset)

cv_results = cross_validate(model2, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)



Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.6355  1.6361  1.6415  1.6322  1.6366  1.6364  0.0030  
MAE (testset)     1.2615  1.2644  1.2642  1.2617  1.2645  1.2633  0.0014  
Fit time          6.66    6.84    6.69    6.71    6.75    6.73    0.06    
Test time         0.51    0.51    0.45    0.52    0.85    0.57    0.14    


In [8]:
#get predictions for testset and seperate my predictions
#may need to run previous module multiple times to ensure 999999 is in testset
predictions = model2.test(testset)
my_predictions = [pred for pred in predictions if pred.uid == 999999]

print(len(my_predictions))

#calc rmse and mae for my predictions
my_rmse = accuracy.rmse(my_predictions)
my_mae = accuracy.mae(my_predictions)

#78 out of 228 predictions evaluated

78
RMSE: 1.0587
MAE:  0.6416
